In [80]:
from typing import Dict, Generator

# Fibonacci sequence

$$ 
fib(n) = fib(n - 1) + fib(n - 2)
$$

Recursion

In [30]:
def fib1(n: int) -> int:
    if n < 2:
        return n
    return fib1(n - 1) + fib1(n - 2)

In [32]:
%timeit fib1(5)

1.32 µs ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Memoization

In [38]:
n = 50

In [76]:
# use a wrapper function to clear the cache so %timeit is correct
def wrapper(n):
    memo: Dict[int, int] = {0: 0, 1: 1}

    def fib3(n: int) -> int:
        if n not in memo:
            memo[n] = fib3(n - 1) + fib3(n - 2)
        return memo[n]
    
    return fib3(n)

In [77]:
%timeit wrapper(n)

15.5 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Automatic memoization

In [78]:
from functools import lru_cache

In [79]:
def wrapper(n):
    @lru_cache(maxsize=None)
    def fib4(n: int) -> int:
        if n < 2:
            return n
        return fib4(n - 1) + fib4(n - 2)
    return fib4(n)

In [73]:
%timeit wrapper(n)

18.1 µs ± 194 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Iterative approach

In [74]:
def fib5(n: int) -> int:
    if n == 0:
        return n
    last: int = 0
    next: int = 1
    for _ in range(1, n):
        last, next = next, last + next
    return next

In [75]:
%timeit fib5(n)

2.08 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Use Generator

In [82]:
def fib6(n: int) -> Generator[int, None, None]:
    yield 0
    if n > 0: yield 1
    last: int = 0
    next: int = 1
    for _ in range(1, n):
        last, next = next, last + next
        yield next

In [83]:
for i in fib6(5):
    print(i)

0
1
1
2
3
5
